In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
# Upload and Load Data
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

Saving realtor-data.zip.csv to realtor-data.zip.csv


In [3]:
# Shape and first 5 rows
print(f"Shape: {df.shape}")
print(df.head())

# Data types
print("\n Data Types ")
print(df.dtypes)

#  Missing values
print("\n Missing Values ")
print(df.isnull().sum())

# Fourth output - Descriptive statistics
print("\n Descriptive Statistics")
print(df.describe(include='all'))

Shape: (2226382, 12)
   brokered_by    status     price  bed  bath  acre_lot     street  \
0     103378.0  for_sale  105000.0  3.0   2.0      0.12  1962661.0   
1      52707.0  for_sale   80000.0  4.0   2.0      0.08  1902874.0   
2     103379.0  for_sale   67000.0  2.0   1.0      0.15  1404990.0   
3      31239.0  for_sale  145000.0  4.0   2.0      0.10  1947675.0   
4      34632.0  for_sale   65000.0  6.0   2.0      0.05   331151.0   

         city        state  zip_code  house_size prev_sold_date  
0    Adjuntas  Puerto Rico     601.0       920.0            NaN  
1    Adjuntas  Puerto Rico     601.0      1527.0            NaN  
2  Juana Diaz  Puerto Rico     795.0       748.0            NaN  
3       Ponce  Puerto Rico     731.0      1800.0            NaN  
4    Mayaguez  Puerto Rico     680.0         NaN            NaN  

 Data Types 
brokered_by       float64
status             object
price             float64
bed               float64
bath              float64
acre_lot          

In [4]:
# Handle Missing Values
print("\nMissing values before cleaning:")
print(df.isna().sum())

# Fill missing categorical data
df['state'] = df['state'].fillna('Unknown')
df['city'] = df['city'].fillna('Unknown')

# Fill numeric columns with median
numeric_cols = ['brokered_by', 'price', 'bed', 'bath', 'acre_lot',
                'street', 'zip_code', 'house_size']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# Convert prev_sold_date to datetime and fill missing with distant past
df['prev_sold_date'] = pd.to_datetime(df['prev_sold_date'], errors='coerce')
df['prev_sold_date'] = df['prev_sold_date'].fillna(pd.to_datetime('1900-01-01'))


Missing values before cleaning:
brokered_by         4533
status                 0
price               1541
bed               481317
bath              511771
acre_lot          325589
street             10866
city                1407
state                  8
zip_code             299
house_size        568484
prev_sold_date    734297
dtype: int64


In [6]:
# Clean Outliers and Impossible Values
# Remove rows with price = 0 or extremely high
df = df[(df['price'] > 1000) & (df['price'] < 10_000_000)]

# Cap unrealistic bedroom/bathroom counts
df['bed'] = df['bed'].clip(1, 10)
df['bath'] = df['bath'].clip(1, 8)

# Remove properties with lot size > 1000 acres
df = df[df['acre_lot'] <= 1000]


In [7]:
df['price_per_bed'] = df['price'] / df['bed']

In [8]:
current_year = pd.to_datetime('now').year
df['years_since_last_sale'] = current_year - df['prev_sold_date'].dt.year

In [9]:
# Convert ID-like columns to integers
df['brokered_by'] = df['brokered_by'].astype(int)
df['street'] = df['street'].astype(int)

# Convert zip_code to string with leading zeros
df['zip_code'] = df['zip_code'].astype(int).astype(str).str.zfill(5)

In [10]:
print("\nMissing values after cleaning:")
print(df.isna().sum())
print(f"\nFinal shape: {df.shape}")
print("\nSample cleaned data:")
print(df.head())


Missing values after cleaning:
brokered_by              0
status                   0
price                    0
bed                      0
bath                     0
acre_lot                 0
street                   0
city                     0
state                    0
zip_code                 0
house_size               0
prev_sold_date           0
price_per_bed            0
years_since_last_sale    0
dtype: int64

Final shape: (2219332, 14)

Sample cleaned data:
   brokered_by    status     price  bed  bath  acre_lot   street        city  \
0       103378  for_sale  105000.0  3.0   2.0      0.12  1962661    Adjuntas   
1        52707  for_sale   80000.0  4.0   2.0      0.08  1902874    Adjuntas   
2       103379  for_sale   67000.0  2.0   1.0      0.15  1404990  Juana Diaz   
3        31239  for_sale  145000.0  4.0   2.0      0.10  1947675       Ponce   
4        34632  for_sale   65000.0  6.0   2.0      0.05   331151    Mayaguez   

         state zip_code  house_size prev_sold_

In [11]:
# 8. Save Cleaned Data
clean_file = 'cleaned_housing_data.csv'
df.to_csv(clean_file, index=False)
files.download(clean_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>